# <span style="color:red">IMPORTING PACKAGES</span>

In [ ]:
# import requests
import pymongo
import random
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy import exc
from sqlalchemy import inspect

<span style="color:blue">The below code imports the pymongo module, creates a connection to the "dap_uat" MongoDB database, and then identifies the "NYPD_Arrest_Data" collection as the one to deal with.</span>


In [24]:
# import the pymongo library
import pymongo

# create a connection to the MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")

# specify the name of the database
my_database = client["dap_uat"]

# specify the name of the collection
collection = my_database["NYPD_Arrest_Data"]



<span style="color:blue">The below code is to drop the entire collection to remove old data and print a message indicating that the old data has been dropped.</span>

In [25]:
# Dropping the entire collection to remove old data
collection.drop()
print("Dropped old data")

Dropped old data


<span style="color:blue">The "json" module is imported in this code below, which then makes use of it to read data into the "data" Python variable from the "NYPD Arrest Data.json" JSON file.</span>

In [26]:
import json

# open the JSON file and read its contents
with open("NYPD Arrest Data.json", "r") as file:
    data = json.load(file)

<span style="color:blue">The below code extracts column names and data rows from a JSON object called "data"</span>

In [27]:
# Extracting column names and data rows
columns = data["meta"]["view"]["columns"]
column_names = ["_".join(col["name"].split()) for col in columns]  # Adjust column names as desired
rows = data["data"]

<span style="color:blue"> The below code removes the excess rows and trims it down to 10,000 rows.</span>

In [28]:
# Randomly deleting around 100,000 rows
rows_to_delete = random.sample(rows, 170000)
for row in rows_to_delete:
    collection.delete_one({"_id": row[0]})  # Assuming first element in row is the unique identifier

# Trim the dataset to 10,000 rows
rows = rows[:10000]

print("Randomly deleted rows and Trimmed Data")

Randomly deleted rows and Trimmed Data


<span style="color:blue"> The below code is to insert the data into MongoDB Atlas</span>

In [29]:
# Insert remaining data into MongoDB
for row in rows:
    # Create dictionary for each row
    cdc_record = {column_names[i]: row[i] for i in range(len(row))}
    # Insert record into MongoDB
    collection.insert_one(cdc_record)

print("Data inserted into MongoDB")


Data inserted into MongoDB


# <span style="color:red">Data Cleaning and Transformation Process</span>

<span style="color:blue">The below codes retrieves data from a MongoDB database, transforms it into a pandas DataFrame, renames certain columns, deletes others, replaces any missing values with "0," and then outputs the number of null values in the finished DataFrame and then the processed data is converted to a structured format CSV .</span>

In [58]:
# Fetch data from MongoDB and convert to DataFrame
df = pd.DataFrame(list(collection.find()))


# Null values Columns dropping
columns_to_drop = ["_id","New_Georeferenced_Column","Longitude","Latitude","Y_COORD_CD","X_COORD_CD","KY_CD","LAW_CODE","ARREST_KEY","ARREST_DATE","PD_CD","updated_meta", "created_meta", "LAW_CAT_CD","sid","id","position","created_at","created_meta","updated_at","meta"]
df.drop(columns_to_drop, axis=1, inplace=True)

new_column_names = {'PD_DESC':'pd_decription','OFNS_DESC':'offense_decription','ARREST_BORO':'arrest_boro','ARREST_PRECINCT':'arrest_precinct','JURISDICTION_CODE':'jurisdiction_code','AGE_GROUP':'age_group','PERP_SEX': 'perp_sex','PERP_RACE': 'perp_race','Community_Districts':'community_districts','Borough_Boundaries':'borough_boundaries','City_Council_Districts':'city_council_districts','Police_Precincts':'police_precincts','Zip_Codes':'zip_codes'}
df.rename(columns=new_column_names, inplace=True)

# Clean the data
# Filling in missing values with a default value
df.fillna("0", inplace=True)


# Check for null values in DataFrame
print(df.isnull().sum())

pd_decription             0
offense_decription        0
arrest_boro               0
arrest_precinct           0
jurisdiction_code         0
age_group                 0
perp_sex                  0
perp_race                 0
community_districts       0
borough_boundaries        0
city_council_districts    0
police_precincts          0
zip_codes                 0
dtype: int64


In [59]:
df.head()

,pd_decription,offense_decription,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,community_districts,borough_boundaries,city_council_districts,police_precincts,zip_codes
0,JOSTLING,JOSTLING,Q,110,1,25-44,M,BLACK,67,3,21,68,14510
1,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY,K,69,0,25-44,M,BLACK,5,2,25,42,13827
2,STRANGULATION 1ST,FELONY ASSAULT,K,61,0,65+,M,WHITE,32,2,15,36,18183
3,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,M,28,0,45-64,M,BLACK,18,4,36,18,12424
4,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY,K,76,0,45-64,M,BLACK,68,2,38,54,16865


In [60]:
df.to_csv('cleaned_nypd.csv',index=False)

In [61]:
cleaned_data_df = pd.read_csv("cleaned_nypd.csv")

# <span style="color:red">Importing the processed data to Postgresql Database</span>

<span style="color:blue"> The below code is to create tables in the database related to the dataset</span>

In [63]:
try:
    cur = conn.cursor() # Create a cursor object for executing SQL queries.
    # SQL query to create the table with column names and data types.
    cur.execute('''
        CREATE TABLE IF NOT EXISTS nypd_arrest_data(
            pd_decription TEXT,
            offense_decription TEXT,
            arrest_boro TEXT,
            arrest_precinct TEXT,
            jurisdiction_code TEXT,
            age_group TEXT,
            perp_sex TEXT,
            perp_race TEXT,
            community_districts TEXT,
            borough_boundaries TEXT,
            city_council_districts TEXT,
            police_precincts TEXT,
            zip_codes TEXT
        );
    ''')
    conn.commit() # Commit the transaction.
    print("Table created successfully!")
except psycopg2.Error as e:
    print("Error creating table:", e)
    conn.rollback() # Rollback the transaction in case of an error.
finally:
    cur.close() # Close the cursor.

Table created successfully!


In [66]:
try:
    engine = create_engine(f'postgresql://{user}:{password}@{host}:5432/{dbname}')
    inspector = inspect(engine)
    # Check if the table already contains data, if so, delete all the rows. 
    table_name = 'nypd_arrest_data'
    if table_name in inspector.get_table_names():
        with conn.cursor() as cursor:
            cursor.execute(f"DELETE FROM nypd_arrest_data")
            conn.commit()  # Commit the DELETE statement
            #print(f"All rows deleted from table '{table_name}'.")
    cleaned_data_df.to_sql('nypd_arrest_data', engine, if_exists='append', index=False)
    engine.dispose() # Dispose the engine after data is loaded.
    print("Data loaded to PostgreSQL successfully!")
except exc.SQLAlchemyError as e:
    print("Error loading data to PostgreSQL:", e)

Data loaded to PostgreSQL successfully!


In [67]:
# Function to extract data from PostgreSQL table and load it into a DataFrame
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    query = 'SELECT * FROM nypd_arrest_data'
    df_from_postgresdb = pd.read_sql(query, conn)
    
    print("Data loaded from PostgreSQL to DataFrame successfully!")
except Exception as e:
    print("Error loading data from PostgreSQL to DataFrame:", e)

Data loaded from PostgreSQL to DataFrame successfully!


/var/folders/pp/93wc99kj26b3hg5_2y8sxlpm0000gn/T/ipykernel_5604/46277063.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_from_postgresdb = pd.read_sql(query, conn)


In [68]:
df_from_postgresdb.head()

,pd_decription,offense_decription,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,community_districts,borough_boundaries,city_council_districts,police_precincts,zip_codes
0,JOSTLING,JOSTLING,Q,110,1,25-44,M,BLACK,67,3,21,68,14510
1,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY,K,69,0,25-44,M,BLACK,5,2,25,42,13827
2,STRANGULATION 1ST,FELONY ASSAULT,K,61,0,65+,M,WHITE,32,2,15,36,18183
3,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,M,28,0,45-64,M,BLACK,18,4,36,18,12424
4,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY,K,76,0,45-64,M,BLACK,68,2,38,54,16865
